<a href="https://colab.research.google.com/github/YacineBenameur/Data-Jobs-Analysis-in-France/blob/main/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Specify openai api key

In [ ]:
openai_api_key = "YOUR_openai_api_key"

# Import libraries

In [ ]:

import pandas as pd
import plotly.express as px
data_path = 'dataset.csv'
data = pd.read_csv(data_path)

# Cleaning

## Basic cleaning

In [ ]:
# Convert publication_date to datetime
data['publication_date'] = pd.to_datetime(data['publication_date'], format='%d/%m/%Y', errors='coerce')

# Filter out data before November 22, 2024
# Deletes 3 jobs
data = data[data['publication_date'] >= '2024-11-22']

# Replace rows where 'contract_type' contains 'Freelance' or 'Permanent'
data['contract_type'] = data['contract_type'].str.replace(r'.*Freelance.*', 'Freelance', regex=True)
data['contract_type'] = data['contract_type'].str.replace(r'.*Permanent.*', 'CDI', regex=True)

# Remove rows where 'contract_type' contains 'Stagiaire'
data = data[~data['contract_type'].str.contains('Stagiaire', na=False)]

# Replace NaN values in 'salaire_min' with values from 'salaire_max' and vice versa
data['salaire_min'] = data['salaire_min'].fillna(data['salaire_max'])
data['salaire_max'] = data['salaire_max'].fillna(data['salaire_min'])

# Add average salary
data['avg_salary'] = data[['salaire_min', 'salaire_max']].mean(axis=1)

# Filter the dataset to exclude rows where 'annual' is True, 'contract_type' is 'CDI', and 'avg_salary' is <= 2000
# Deletes 15 jobs
data = data[~((data['annual'] == True) & (data['contract_type'] == 'CDI') & (data['salaire_min'] <= 10000))]

# Remove rows where 'contract_type' is 'Freelance' and 'avg_salary' is greater than 10000
data = data[~((data['contract_type'] == 'Freelance') & (data['avg_salary'] > 10000))]




## Sectors standardization

In [ ]:
# Define mapping
sector_mapping = {
    'BTP': 'Construction',
    'Construction': 'Construction',
    'Aéronautique': 'Aerospace',
    'Aerospace': 'Aerospace',
    'Auto': 'Automotive',
    'Automotive': 'Automotive',
    'Automobile': 'Automotive',
    'Informatique': 'IT',
    'IT': 'IT',
    'Technology': 'Technology',
    'Tech': 'Technology',
    'Media': 'Media',
    'Média': 'Media',
    'Healthcare': 'Health',
    'Santé': 'Health',
    'Telecom': 'Telecommunications',
    'Telecommunications': 'Telecommunications',
    'Agriculture': 'Agriculture',
    'Agro-alimentaire': 'Agro-food',
    'Agroalimentaire': 'Agro-food',
    'Agro-food': 'Agro-food',
    'Energie': 'Energy',
    'Energy': 'Energy',
    'Environment': 'Environment',
    'Environmental': 'Environment',
    'Tourism': 'Tourism',
    'Tourisme': 'Tourism',
    'Education': 'Education',
    'Teaching': 'Education',
    'Enseignement': 'Education',
    'Pharmaceutical': 'Pharmaceuticals',
    'Pharmaceuticals': 'Pharmaceuticals',
    'Real Estate': 'Real Estate',
    'Immobilier': 'Real Estate',
    'Industry': 'Industry',
    'Industrie': 'Industry',
    'Public': 'Public',
    'Service public': 'Public',
    'Consulting': 'Consulting',
    'Logistics': 'Logistics',
    'Defense': 'Defense',
    'Research': 'Research',
    'Engineering': 'Engineering',
    'Internet': 'Internet',
    'Data': 'Data',
    'Commerce': 'Commerce',
    'Distribution': 'Commerce',
    'Manufacturing': 'Manufacturing',
    'Transport': 'Transport',
    'Food': 'Agro-food'
}

# Standardize the sector column
data['sector'] = data['sector'].map(sector_mapping)


## Categorize data jobs

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
from openai import OpenAI
import pandas as pd

# Define the schema for the output
class JobCategorization(BaseModel):
    title: str
    concepts: Optional[List[str]] = []
    categories: List[str] = Field(..., description="List of applicable categories for the job")

# Function to query OpenAI API
def get_openai_categorization(client: OpenAI, title: str, concepts: Optional[List[str]]) -> JobCategorization:
    concepts_str = ", ".join(concepts) if concepts else ""
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that categorizes job titles and associated concepts into predefined categories."
        },
        {
            "role": "user",
            "content": f"""
Catégories finales pour les métiers liés à la data :

Data Analyst
Inclut également les rôles liés à la Business Intelligence (BI).
Exemples :
"Data Analyst", "Business Data Analyst", "BI Developer".

Data Scientist
Inclut les rôles liés au machine learning et à l’intelligence artificielle (ML Engineer).
Exemples :
"Data Scientist", "Machine Learning Engineer", "AI Researcher".

Data Engineer
Gestion des pipelines, infrastructure de données, ETL, et architecture.
Exemples :
"Data Engineer", "Big Data Engineer", "Architecte Data".

Project Manager (Data)
Inclut les rôles de chef de projet et de product owner en data.
Exemples :
"Chef de Projet Data", "Product Owner Data", "Data Project Manager".

Data Governance & Quality
Pour les métiers axés sur la gouvernance, la gestion, et la qualité des données.
Exemples :
"Data Quality Manager", "Consultant Data Gouvernance", "Data Steward".

Other
Tous les rôles non liés à la data.

Basé sur le titre du poste et les concepts associés, catégorisez le poste dans une ou plusieurs des catégories ci-dessus. Certains rôles peuvent chevaucher plusieurs catégories.

Title: {title}
Concepts: {concepts_str}

Retournez les catégories sous forme d'un JSON strictement formaté :
{{
    "title": "{title}",
    "concepts": [{concepts_str}],
    "categories": ["Category1", "Category2"]
}}
"""
        }
    ]

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    # Access the content of the response correctly
    structured_data = response.choices[0].message.content.strip("```json").strip("```").strip()
    return JobCategorization.parse_raw(structured_data)

# Function to process a DataFrame
def categorize_jobs(client: OpenAI, data: pd.DataFrame) -> pd.DataFrame:
    def categorize_row(row):
        title = row['title']
        concepts = row['concepts'] if isinstance(row['concepts'], list) else []
        job_categorization = get_openai_categorization(client, title, concepts)
        return job_categorization.categories

    data['categories'] = data.apply(categorize_row, axis=1)
    return data

# Example usage with your OpenAI client
client = OpenAI(api_key=openai_api_key)




In [ ]:
categorized_data = categorize_jobs(client, data)
categorized_data


# Save cleaned data

In [ ]:
categorized_data.to_csv("cleaned_dataset.csv", index=False)


In [ ]:
# Calculate the ratio of rows with "Other" as a category
total_rows = len(data)
other_rows = data['categories'].apply(lambda x: "Other" in x).sum()
ratio_other = other_rows / total_rows

print(f"Ratio of rows with 'Other' as a category: {ratio_other:.2%}")
